In [80]:
from automata import *
import json

In [81]:
Q = ['q0', 'q1', 'q2']#'q1', 'q2'
A = ['a1', 'a2', ] #'a3'
B = ['0', '1']
phi = {
    ('q0', A[0]): 'q0',
    ('q0', A[1]): 'q1',#'q1',
    #('q0', A[2]): 'q1',

    ('q1', A[0]): 'q2',
    ('q1', A[1]): 'q2',
    #('q1', A[2]): 'q2',
#
    ('q2', A[0]): 'q2',  
    ('q2', A[1]): 'q2', 
    #('q2', A[2]): 'q2', 


}

psi = {
    ('q0', A[0]): B[0],
    ('q0', A[1]): B[1],
    #('q0', A[2]): B[1],
#
    ('q1', A[0]): B[0],
    ('q1', A[1]): B[1],
    #('q1', A[2]): B[1],
#
    ('q2', A[0]): B[0],
    ('q2', A[1]): B[0],
    #('q2', A[2]): B[0],

}
q_start = 'q0'

In [82]:
schema_encoding = {

    'a1': '00000',
    'a2': '000',

}


In [83]:
def load_automaton_from_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)

    Q = data.get('Q')
    A = data.get('A')
    B = data.get('B')
    phi_raw = data.get('phi')
    psi_raw = data.get('psi')
    q_start = data.get('q_start')

  
    if not isinstance(Q, list) or not all(isinstance(q, str) for q in Q):
        raise ValueError("Q must be a list of strings.")
    if not isinstance(A, list) or not all(isinstance(a, str) for a in A):
        raise ValueError("A must be a list of strings.")
    if B != ['0', '1']:
        raise ValueError("B must be exactly ['0', '1'].")
    if q_start not in Q:
        raise ValueError("q_start must be one of the states in Q.")

    phi = {}
    psi = {}
    expected_keys = {(q, a) for q in Q for a in A}

    for key_str, val in phi_raw.items():
        key = tuple(k.strip() for k in key_str.strip("()").split(","))
        if len(key) != 2 or key[0] not in Q or key[1] not in A:
            raise ValueError(f"Invalid key in phi: {key}")
        if val not in Q:
            raise ValueError(f"Invalid state target in phi({key}): {val}")
        phi[(key[0], key[1])] = val

    for key_str, val in psi_raw.items():
        key = tuple(k.strip() for k in key_str.strip("()").split(","))
        if len(key) != 2 or key[0] not in Q or key[1] not in A:
            raise ValueError(f"Invalid key in psi: {key}")
        if val not in B:
            raise ValueError(f"Invalid output symbol in psi({key}): {val}")
        psi[(key[0], key[1])] = val

    for q in Q:
        for a in A:
            if (q, a) not in phi:
                raise ValueError(f"Missing transition in phi for state {q} and symbol {a}")
            if (q, a) not in psi:
                raise ValueError(f"Missing transition in psi for state {q} and symbol {a}")

    return Q, A, B, phi, psi, q_start

Q, A, B, phi, psi, q_start = load_automaton_from_json('../data/automata.json')
print(Q, A, B, phi, psi, q_start)


['q0', 'q1', 'q2'] ['a1', 'a2'] ['0', '1'] {('q0', 'a1'): 'q0', ('q0', 'a2'): 'q1', ('q1', 'a1'): 'q2', ('q1', 'a2'): 'q2', ('q2', 'a1'): 'q2', ('q2', 'a2'): 'q2'} {('q0', 'a1'): '1', ('q0', 'a2'): '1', ('q1', 'a1'): '0', ('q1', 'a2'): '1', ('q2', 'a1'): '0', ('q2', 'a2'): '0'} q0


In [ ]:
def load_schema_encoding(filename, A):
    with open(filename, 'r') as f:
        schema_encoding = json.load(f)

    if not isinstance(schema_encoding, dict):
        raise ValueError("Encoding must be a dictionary.")

    keys = set(schema_encoding.keys())
    expected = set(A)

    if keys != expected:
        missing = expected - keys
        extra = keys - expected
        msg = []
        if missing:
            msg.append(f"Missing keys: {missing}")
        if extra:
            msg.append(f"Unexpected keys: {extra}")
        raise ValueError("Invalid encoding keys. " + "; ".join(msg))

    for k, v in schema_encoding.items():
        if not isinstance(v, str):
            raise ValueError(f"Encoding for {k} must be a string.")

    return schema_encoding

schema_encoding = load_schema_encoding('../data/schema_encoding.json', A)
print(schema_encoding)

{'a1': '00000', 'a2': '000'}


In [ ]:
def number_of_fault_diagnostics(schema_encoding, A, B, Q, phi, psi, q_start): 

    if not is_minimized(Q, A, phi, psi, q_start) or is_automata_ambiguous(schema_encoding, A, B, Q, phi, psi, q_start):
        print('initial automata is not minimized or ambiguous')
        return 0

    faults_count = 0
    for key in psi:
        copy_psi = psi.copy()
        copy_psi[key] = B[0] if copy_psi[key] == B[1] else B[1]

        not_minimized = not is_minimized(Q, A, phi, copy_psi, q_start)
        ambigious = is_automata_ambiguous(schema_encoding, A, B, Q, phi, copy_psi, q_start)
        if  not_minimized or ambigious:
            print("not_minimized = ", not_minimized, '; ambigious = ', ambigious, '; key = ', key)
            faults_count += 1
            
    return faults_count


In [86]:
def is_fault_happened(schema_encoding, A, B, Q, phi, psi, q_start):
    is_initial_correct = is_minimized(Q, A, phi, psi, q_start) and not is_automata_ambiguous(schema_encoding, A, B, Q, phi, psi, q_start)
    brake_to_create_correct = set()
    for key in psi:
        copy_psi = psi.copy()
        copy_psi[key] = B[0] if copy_psi[key] == B[1] else B[1]

        if is_minimized(Q, A, phi, copy_psi, q_start) or not is_automata_ambiguous(schema_encoding, A, B, Q, phi, copy_psi, q_start):
            brake_to_create_correct.add(key)
                     
    brake_to_create_correct_len = len(brake_to_create_correct)

    if is_initial_correct:
        if brake_to_create_correct_len == 0:
            print('no fault happened')
            return False
        else:
            print('as initial automata is correct and we found broken automata those are also correct, we cannot say if fault happened or not')
            return None
    else:
        if brake_to_create_correct_len == 0:
            print('invalid input automata')
            return None
        elif brake_to_create_correct_len == 1:
            print('we can say that fault happened, and where it is happened: ', brake_to_create_correct)
            return True
        else:
            print('we can say that fault happened, but no information about where it could be (more than 1 correct broken automata found): ', brake_to_create_correct)
            return True


In [87]:
res = number_of_fault_diagnostics(schema_encoding, A, B, Q, phi, psi, q_start)
print('number of faults that can be diagnosed:', res)

not_minimized =  False ; ambigious =  True ; key =  ('q1', 'a1')
not_minimized =  True ; ambigious =  False ; key =  ('q1', 'a2')
not_minimized =  False ; ambigious =  True ; key =  ('q2', 'a1')
not_minimized =  True ; ambigious =  True ; key =  ('q2', 'a2')
number of faults that can be diagnosed: 4


In [88]:
res = is_fault_happened(schema_encoding, A, B, Q, phi, psi, q_start)
print(res)

as initial automata is correct and we found broken automata those are also correct, we cannot say if fault happened or not
None
